# Rs_Bernstein Correction
Example of the BernsteinCorrection utility in RooStats.

The idea is that one has a distribution coming either from data or Monte Carlo
(called "reality" in the macro) and a nominal model that is not sufficiently
flexible to take into account the real distribution.  One wants to take into
account the systematic associated with this imperfect modeling by augmenting
the nominal model with some correction term (in this case a polynomial).
The BernsteinCorrection utility will import into your workspace a corrected model
given by nominal(x) * poly_N(x), where poly_N is an n-th order polynomial in
the Bernstein basis.  The degree N of the polynomial is chosen by specifying the tolerance
one has in adding an extra term to the polynomial.
The Bernstein basis is nice because it only has positive-definite terms
and works well with PDFs.
Finally, the macro makes a plot of:
 - the data (drawn from 'reality'),
 - the best fit of the nominal model (blue)
 - and the best fit corrected model.




**Author:** Kyle Cranmer  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:24 AM.</small></i>

In [1]:
%%cpp -d
#include "RooDataSet.h"
#include "RooRealVar.h"
#include "RooConstVar.h"
#include "RooBernstein.h"
#include "TCanvas.h"
#include "RooAbsPdf.h"
#include "RooFit.h"
#include "RooFitResult.h"
#include "RooPlot.h"
#include <string>
#include <vector>
#include <stdio.h>
#include <sstream>
#include <iostream>

#include "RooProdPdf.h"
#include "RooAddPdf.h"
#include "RooGaussian.h"
#include "RooNLLVar.h"
#include "RooMinuit.h"
#include "RooProfileLL.h"
#include "RooWorkspace.h"

#include "RooStats/BernsteinCorrection.h"

Use this order for safety on library loading

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;
using namespace RooStats;

____________________________________

Set range of observable

In [3]:
Double_t lowRange = -1, highRange = 5;

Make a roorealvar for the observable

In [4]:
RooRealVar x("x", "x", lowRange, highRange);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



True model

In [5]:
RooGaussian narrow("narrow", "", x, RooConst(0.), RooConst(.8));
RooGaussian wide("wide", "", x, RooConst(0.), RooConst(2.));
RooAddPdf reality("reality", "", RooArgList(narrow, wide), RooConst(0.8));

RooDataSet *data = reality.generate(x, 1000);

Nominal model

In [6]:
RooRealVar sigma("sigma", "", 1., 0, 10);
RooGaussian nominal("nominal", "", x, RooConst(0.), sigma);

RooWorkspace *wks = new RooWorkspace("myWorksspace");

wks->import(*data, Rename("data"));
wks->import(nominal);

if (TClass::GetClass("ROOT::Minuit2::Minuit2Minimizer")) {
   // use Minuit2 if ROOT was built with support for it:
   ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");
}

[#1] INFO:ObjectHandling -- RooWorkspace::import(myWorksspace) importing dataset realityData
[#1] INFO:ObjectHandling -- RooWorkSpace::import(myWorksspace) changing name of dataset from  realityData to data
[#1] INFO:ObjectHandling -- RooWorkspace::import(myWorksspace) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(myWorksspace) importing RooGaussian::nominal
[#1] INFO:ObjectHandling -- RooWorkspace::import(myWorksspace) importing RooConstVar::0
[#1] INFO:ObjectHandling -- RooWorkspace::import(myWorksspace) importing RooRealVar::sigma


The tolerance sets the probability to add an unnecessary term.
 lower tolerance will add fewer terms, while higher tolerance
 will add more terms and provide a more flexible function.

In [7]:
Double_t tolerance = 0.05;
BernsteinCorrection bernsteinCorrection(tolerance);
Int_t degree = bernsteinCorrection.ImportCorrectedPdf(wks, "nominal", "x", "data");

if (degree < 0) {
   Error("rs_bernsteinCorrection", "Bernstein correction failed ! ");
   return;
}

cout << " Correction based on Bernstein Poly of degree " << degree << endl;

RooPlot *frame = x.frame();
data->plotOn(frame);

BernsteinCorrection::ImportCorrectedPdf -  Doing initial Fit with nominal model 
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_nominal_FOR_OBS_x with 0 entries
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(corrected_clone_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(corrected_clone_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
[#1] INFO:NumericInte

Plot the best fit nominal model in blue

In [8]:
TString minimType = ROOT::Math::MinimizerOptions::DefaultMinimizerType();
nominal.fitTo(*data, PrintLevel(0), Minimizer(minimType));
nominal.plotOn(frame);

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 1216.77793416266263
Edm   = 4.16187321843267985e-07
Nfcn  = 19
sigma	  = 1.18138	 +/-  0.0315451	(limited)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization


Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


Plot the best fit corrected model in red

In [9]:
RooAbsPdf *corrected = wks->pdf("corrected");
if (!corrected)
   return;

Fit corrected model

In [10]:
corrected->fitTo(*data, PrintLevel(0), Minimizer(minimType));
corrected->plotOn(frame, LineColor(kRed));

[#1] INFO:NumericIntegration -- RooRealIntegral::init(corrected_clone_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_corrected_FOR_OBS_x with 0 entries
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 3500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 1182.56771175955873
Edm   = 0.000104308534890513858
Nfcn  = 184
c_1	  = 3.1837	 +/-  0.834348	(limited)
c_2	  = 1.2322e-05	 +/-  3.09822	(limited)
c_3	  = 1.61168e-06	 +/-  1.52589	(limited)
c_4	  = 0.971334	 +/-  2.5239	(limited)
c_5	  = 0.200166	 +/-  75.522	(limited)
c_6	  = 10.5071	 +/-  22.97	(limited)
sigma	  = 1.26617	 +/-  0.232029	(limited)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(corrected_clone_Int[x]) using num

Info: VariableMetricBuilder: Tolerance is not sufficient, continue the minimization
Info in Current  Edm is : edm = 0.00156657
Info in Required Edm is : edmval = 0.001
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


Plot the correction term (* norm constant) in dashed green
 should make norm constant just be 1, not depend on binning of data

In [11]:
RooAbsPdf *poly = wks->pdf("poly");
if (poly)
   poly->plotOn(frame, LineColor(kGreen), LineStyle(kDashed));

This is a switch to check the sampling distribution
 of -2 log LR for two comparisons:
 the first is for n-1 vs. n degree polynomial corrections
 the second is for n vs. n+1 degree polynomial corrections
 Here we choose n to be the one chosen by the tolerance
 criterion above, eg. n = "degree" in the code.
 Setting this to true is takes about 10 min.

In [12]:
bool checkSamplingDist = true;
int numToyMC = 20; // increase this value for sensible results

TCanvas *c1 = new TCanvas();
if (checkSamplingDist) {
   c1->Divide(1, 2);
   c1->cd(1);
}
frame->Draw();
gPad->Update();

if (checkSamplingDist) {
   // check sampling dist
   ROOT::Math::MinimizerOptions::SetDefaultPrintLevel(-1);
   TH1F *samplingDist = new TH1F("samplingDist", "", 20, 0, 10);
   TH1F *samplingDistExtra = new TH1F("samplingDistExtra", "", 20, 0, 10);
   bernsteinCorrection.CreateQSamplingDist(wks, "nominal", "x", "data", samplingDist, samplingDistExtra, degree,
                                           numToyMC);

   c1->cd(2);
   samplingDistExtra->SetLineColor(kRed);
   samplingDistExtra->Draw();
   samplingDist->Draw("same");
}

made pdfs, make toy generator
on toy 0
on toy 1
on toy 2
on toy 3
on toy 4
on toy 5
on toy 6
on toy 7
on toy 8
on toy 9
on toy 10
on toy 11
on toy 12
on toy 13
on toy 14
on toy 15
on toy 16
on toy 17
on toy 18
on toy 19


Draw all canvases 

In [13]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()